In [1]:
# imports
import requests
import pandas as pd
import numpy as np

In [2]:
# gets data from API
def fetchData(url):
    r = requests.get(url)
    return r.json()

In [3]:
# calls fetchData function for chosen URL
general_info_json = fetchData('https://fantasy.premierleague.com/api/bootstrap-static/')
fixtures_json = fetchData('https://fantasy.premierleague.com/api/fixtures/')
#players_detailed_json = fetchData('https://fantasy.premierleague.com/api/element-summary/{' + element_id + '}/')
#gameweek_live_json = fetchData('https://fantasy.premierleague.com/api/event/{' + event_id + '}/live/')

In [4]:
# gets a list of keys of the json
general_info_json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [5]:
# gets a list of keys of the json
fixtures_json[0].keys()

dict_keys(['code', 'event', 'finished', 'finished_provisional', 'id', 'kickoff_time', 'minutes', 'provisional_start_time', 'started', 'team_a', 'team_a_score', 'team_h', 'team_h_score', 'stats', 'team_h_difficulty', 'team_a_difficulty', 'pulse_id'])

In [6]:
#creates data frame
elements_df = pd.DataFrame(general_info_json['elements'])

In [8]:
elements_types_df = pd.DataFrame(general_info_json['element_types'])

In [10]:
elements_df['position'] = elements_df.element_type.map(elements_types_df.set_index('id').singular_name)

In [11]:
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [12]:
fantasy_stats_df = elements_df[['first_name', 'second_name','team','position','selected_by_percent','now_cost','transfers_in','transfers_out','value_season', 'total_points']]

In [14]:
#player_cost_df = elements_df[['id','second_name','cost_change_event','cost_change_event_fall','cost_change_start','now_cost']]
#chance_of_playing_df = elements_df[['id','second_name','chance_of_playing_next_round','chance_of_playing_this_round']]

In [15]:
#final_team = {'GK':0,'GK2':0,'DEF1':0,'DEF2':0,'DEF3':0,'DEF4':0,'DEF5':0,'MID1':0,'MID2':0,'MID3':0,'MID4':0,'MID5':0,'FOR1':0,'FOR2':0,'FOR3':0,}

In [16]:
choosing_team_df = elements_df[['id','first_name','second_name','team','position','now_cost','status','total_points','value_season']]

In [17]:
# removes all players who currently are not 100% available for the next game
choosing_team_df = choosing_team_df[choosing_team_df.status == 'a']

# removes all players who currently has no points
choosing_team_df = choosing_team_df[choosing_team_df.total_points != '0.0']

In [18]:
#choosing_team_df.replace({'element_type' : { 1 : 'GK', 2 : 'DEF', 3 : 'MID' , 4 : 'FOR'}})

In [19]:
# best value GKs
choosing_team_df[choosing_team_df.position == 'Goalkeeper'].sort_values(by=['value_season'], ascending = False).head(10)

,id,first_name,second_name,team,position,now_cost,status,total_points,value_season
73,80,David,Raya Martin,3,Goalkeeper,46,a,30,6.5
487,353,Hugo,Lloris,17,Goalkeeper,55,a,35,6.4
606,475,José,Malheiro de Sá,20,Goalkeeper,50,a,30,6.0
193,146,Vicente,Guaita,7,Goalkeeper,45,a,26,5.8
355,257,Ederson,Santana de Moraes,12,Goalkeeper,60,a,35,5.8
326,231,Alisson,Ramses Becker,11,Goalkeeper,60,a,35,5.8
375,270,David,de Gea,13,Goalkeeper,51,a,28,5.5
38,30,Emiliano,Martínez,2,Goalkeeper,55,a,30,5.5
114,69,Robert,Sánchez,4,Goalkeeper,46,a,25,5.4
33,559,Aaron,Ramsdale,1,Goalkeeper,46,a,24,5.2


In [20]:
# best value DEFs
choosing_team_df[choosing_team_df.position == 'Defender'].sort_values(by=['value_season'], ascending = False).head(10)

,id,first_name,second_name,team,position,now_cost,status,total_points,value_season
84,91,Ethan,Pinnock,3,Defender,46,a,38,8.3
69,76,Pontus,Jansson,3,Defender,46,a,38,8.3
102,56,Shane,Duffy,4,Defender,43,a,35,8.1
354,256,João Pedro Cavaco,Cancelo,12,Defender,62,a,44,7.1
187,527,Trevoh,Chalobah,6,Defender,48,a,32,6.7
165,122,Marcos,Alonso,6,Defender,60,a,39,6.5
360,262,Rúben Santos,Gato Alves Dias,12,Defender,61,a,39,6.4
170,127,Antonio,Rüdiger,6,Defender,58,a,36,6.2
588,432,Fernando,Marçal,20,Defender,46,a,28,6.1
162,119,César,Azpilicueta,6,Defender,60,a,36,6.0


In [21]:
# best value MIDs
choosing_team_df[choosing_team_df.position == 'Midfielder'].sort_values(by=['value_season'], ascending = False).head(10)

,id,first_name,second_name,team,position,now_cost,status,total_points,value_season
242,482,Andros,Townsend,8,Midfielder,56,a,43,7.7
232,173,Abdoulaye,Doucouré,8,Midfielder,56,a,43,7.7
569,419,Saïd,Benrahma,19,Midfielder,66,a,44,6.7
168,125,Mateo,Kovacic,6,Midfielder,52,a,33,6.3
539,399,Ismaila,Sarr,18,Midfielder,63,a,40,6.3
243,484,Demarai,Gray,8,Midfielder,58,a,36,6.2
192,144,Conor,Gallagher,7,Midfielder,57,a,34,6.0
328,233,Mohamed,Salah,11,Midfielder,127,a,70,5.5
230,171,Allan,Marques Loureiro,8,Midfielder,46,a,24,5.2
89,96,Bryan,Mbeumo,3,Midfielder,55,a,27,4.9


In [22]:
# best value FORs
choosing_team_df[choosing_team_df.position == 'Forward'].sort_values(by=['value_season'], ascending = False).head(10)

,id,first_name,second_name,team,position,now_cost,status,total_points,value_season
563,413,Michail,Antonio,19,Forward,80,a,49,6.1
418,307,Allan,Saint-Maximin,14,Forward,68,a,37,5.4
109,63,Neal,Maupay,4,Forward,65,a,32,4.9
71,78,Ivan,Toney,3,Forward,63,a,31,4.9
361,263,Gabriel Fernando,de Jesus,12,Forward,86,a,41,4.8
261,205,Jamie,Vardy,9,Forward,105,a,48,4.6
546,450,Emmanuel,Dennis,18,Forward,53,a,24,4.5
611,583,Hee-Chan,Hwang,20,Forward,56,a,24,4.3
433,315,Teemu,Pukki,15,Forward,59,a,23,3.9
61,337,Danny,Ings,2,Forward,79,a,30,3.8


In [23]:
# best value players
choosing_team_df.sort_values(by=['value_season'], ascending = False).head(10)

,id,first_name,second_name,team,position,now_cost,status,total_points,value_season
84,91,Ethan,Pinnock,3,Defender,46,a,38,8.3
69,76,Pontus,Jansson,3,Defender,46,a,38,8.3
102,56,Shane,Duffy,4,Defender,43,a,35,8.1
232,173,Abdoulaye,Doucouré,8,Midfielder,56,a,43,7.7
242,482,Andros,Townsend,8,Midfielder,56,a,43,7.7
354,256,João Pedro Cavaco,Cancelo,12,Defender,62,a,44,7.1
569,419,Saïd,Benrahma,19,Midfielder,66,a,44,6.7
187,527,Trevoh,Chalobah,6,Defender,48,a,32,6.7
73,80,David,Raya Martin,3,Goalkeeper,46,a,30,6.5
165,122,Marcos,Alonso,6,Defender,60,a,39,6.5


In [24]:
position_points_value_df = elements_df[['status', 'minutes', 'position', 'total_points', 'value_season']]
position_points_value_df['value'] = position_points_value_df.value_season.astype(float)
position_points_value_df = position_points_value_df[position_points_value_df.status != 'u']
position_points_value_df = position_points_value_df[position_points_value_df.minutes != 0]

ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
# total points per position on average
# only including players currently in the premier league and who have played 1 min+
position_points_df_pivot = pd.pivot_table(data=position_points_value_df[['position', 'total_points']],index=['position'])
position_points_df_pivot.sort_values(by=['total_points'], ascending = False)

,total_points
position,
Goalkeeper,19.692308
Forward,14.821429
Defender,13.444444
Midfielder,13.304348


In [27]:
# value per position on average
# only including players currently in the premier league and who have played 1 min+
position_value_df_pivot = pd.pivot_table(data=position_points_value_df[['position', 'value']],index=['position'])
position_value_df_pivot.sort_values(by=['value'], ascending = False)

,value
position,
Goalkeeper,3.892308
Defender,2.729167
Midfielder,2.244565
Forward,2.092857


Goalkeepers seem to get the most points on average (likely skewed due to the lower amount of goalkeepers included compared to other positions) and they are also the best in terms of value.

Forwards on average get a high amount of points but are the worst in terms of value (most high scoring forwards are the most expensive players in the game)

In [36]:
position_points_value_df.groupby('position').size()

position
Defender      144
Forward        56
Goalkeeper     26
Midfielder    184
dtype: int64

The count of players per position shows how little goalkeepers are included in this compared to other positions

In [52]:
# best value FORs
choosing_team_df[choosing_team_df.position == 'Goalkeeper'].sort_values(by=['total_points'], ascending = False).head(2)

,id,first_name,second_name,team,position,now_cost,status,total_points,value_season
355,257,Ederson,Santana de Moraes,12,Goalkeeper,60,a,35,5.8
326,231,Alisson,Ramses Becker,11,Goalkeeper,60,a,35,5.8


In [53]:
choosing_team_df[choosing_team_df.position == 'Defender'].sort_values(by=['total_points'], ascending = False).head(5)

,id,first_name,second_name,team,position,now_cost,status,total_points,value_season
354,256,João Pedro Cavaco,Cancelo,12,Defender,62,a,44,7.1
165,122,Marcos,Alonso,6,Defender,60,a,39,6.5
360,262,Rúben Santos,Gato Alves Dias,12,Defender,61,a,39,6.4
84,91,Ethan,Pinnock,3,Defender,46,a,38,8.3
69,76,Pontus,Jansson,3,Defender,46,a,38,8.3


In [54]:
choosing_team_df[choosing_team_df.position == 'Midfielder'].sort_values(by=['total_points'], ascending = False).head(5)

,id,first_name,second_name,team,position,now_cost,status,total_points,value_season
328,233,Mohamed,Salah,11,Midfielder,127,a,70,5.5
569,419,Saïd,Benrahma,19,Midfielder,66,a,44,6.7
232,173,Abdoulaye,Doucouré,8,Midfielder,56,a,43,7.7
242,482,Andros,Townsend,8,Midfielder,56,a,43,7.7
492,359,Heung-Min,Son,17,Midfielder,100,a,42,4.2


In [55]:
choosing_team_df[choosing_team_df.position == 'Forward'].sort_values(by=['total_points'], ascending = False).head(3)

,id,first_name,second_name,team,position,now_cost,status,total_points,value_season
563,413,Michail,Antonio,19,Forward,80,a,49,6.1
261,205,Jamie,Vardy,9,Forward,105,a,48,4.6
361,263,Gabriel Fernando,de Jesus,12,Forward,86,a,41,4.8
